# **ENGG680 - Introduction to Digital Engineering**
## *Lab Assignment 4: ML Applications - Vehicle Motion State Classification*

## Preliminary: Certificate of Work


*We, the undersigned, certify that this is our own work, which has been done expressly for this course, either without the assistance of any other party or where appropriate we have acknowledged the work of others. Further, we have read and understood the section in the university calendar on plagiarism/cheating/other academic misconduct and we are aware of the implications thereof. We request that the total mark for this assignment be distributed as follows among group members:*

---

In [2]:
import pandas as pd

# Define the data with Member labels, ensuring hours are integers
data = {
    'First Name': ['Musad Rafiouddin', 'Sushant Samir', 'Venkata Prasanna Kumar', 'Prince'],
    'Last Name': ['Saiyed', 'Khedekar', 'Gurugubelli', 'Ifem'],
    'Signature (Full Name, Date)': ['Musad Rafiouddin Saiyed', 'Sushant Samir Khedekar', 'Venkata Prasanna Kumar Gurugubelli', 'Prince Ifem'],
    'Hours': [4, 3, 3, 3],  # Use integer values for Hours
    'Contribution %': ['25%', '25%', '25%', '25%']
}

# Create DataFrame
df = pd.DataFrame(data)

# Set custom index to match "Member" labels
df.index = ['Member 1:', 'Member 2:', 'Member 3:', 'Member 4:']

# Display the DataFrame with style formatting
styled_df = df.style.set_table_styles([
    {'selector': 'th', 'props': [('font-weight', 'bold')]},
    {'selector': 'td', 'props': [('text-align', 'center')]},
    {'selector': 'th.col_heading', 'props': [('text-align', 'center')]},
    {'selector': 'tr:nth-child(even)', 'props': [('background-color', '#f2f2f2')]}
])

styled_df

,First Name,Last Name,"Signature (Full Name, Date)",Hours,Contribution %
Member 1:,Musad Rafiouddin,Saiyed,Musad Rafiouddin Saiyed,4,25%
Member 2:,Sushant Samir,Khedekar,Sushant Samir Khedekar,3,25%
Member 3:,Venkata Prasanna Kumar,Gurugubelli,Venkata Prasanna Kumar Gurugubelli,3,25%
Member 4:,Prince,Ifem,Prince Ifem,3,25%


## Section 1 - Motion Classification Using *All Features* Derived From Inertial Sensor Data (33 Marks)

---

### 1.1 Assignment datasets import (6 Marks)

---

In [3]:
# some library imports
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


First load the following datasets into dataframes.
* *UrbanNav-HK_Whampoa-20210521_sensors.csv* to dataframe variable named *cv_set*. This dataset is to be used for model training and hyper parameter tuning.
* *UrbanNav-HK_TST-20210517_sensors.csv* to dataframe variable named *test_set1*. This is first independent test dataset.
* *2020-03-14-16-45-35.csv* to dataframe variable named *test_set2*. This is the second independent test dataset.

In [6]:
# (2 marks)
# TODO: Load cross validation/training dataset (UrbanNav-HK_Whampoa-20210521_sensors.csv) to 'cv_set' dataframe

# uncomment and compelete below
cv_set = pd.read_csv('/content/drive/MyDrive/engg680_2024_fall/Lab_Assignment4/UrbanNav-HK_Whampoa-20210521_sensors.csv')

# TODO: Load UrbanNav-HK_TST-20210517_sensors.csv to 'test_set1' dataframe
test_set1 = pd.read_csv('/content/drive/MyDrive/engg680_2024_fall/Lab_Assignment4/UrbanNav-HK_TST-20210517_sensors.csv')

# TODO: Load 2020-03-14-16-45-35.csv to 'test_set2' dataframe
test_set2 = pd.read_csv('/content/drive/MyDrive/engg680_2024_fall/Lab_Assignment4/2020-03-14-16-45-35.csv')

For the *cv_set*, look at the dataframe head and info, and print the list of all columns. The *test_set1* and *test_set2* have the same schema as *cv_set*, however, have different number of rows (samples).

In [7]:
# TODO: Display first two rows of cv_set
# (1 mark)

cv_set.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max,motion_state
0,-0.003929,-0.093081,9.789349,-0.002649,0.001406,-0.001391,9.797418,0.022554,8.046094e-07,1.319665e-07,...,0.000003,0.001496,0.000544,0.024992,0.000110,0.000075,0.000078,0.024997,0.000187,rest
1,-0.000966,-0.092417,9.789095,-0.001806,0.000103,-0.001943,9.798815,0.021731,1.032619e-06,1.073067e-07,...,0.000006,0.001865,0.000615,0.024828,0.000124,0.000098,0.000090,0.024829,0.000144,rest


In [8]:
# TODO: Load the info method of cv_set
# (1 mark)

cv_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1534 entries, 0 to 1533
Data columns (total 33 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ax_sum        1534 non-null   float64
 1   ay_sum        1534 non-null   float64
 2   az_sum        1534 non-null   float64
 3   gx_sum        1534 non-null   float64
 4   gy_sum        1534 non-null   float64
 5   gz_sum        1534 non-null   float64
 6   a_sum         1534 non-null   float64
 7   g_sum         1534 non-null   float64
 8   ax_var        1534 non-null   float64
 9   ay_var        1534 non-null   float64
 10  az_var        1534 non-null   float64
 11  gx_var        1534 non-null   float64
 12  gy_var        1534 non-null   float64
 13  gz_var        1534 non-null   float64
 14  a_var         1534 non-null   float64
 15  g_var         1534 non-null   float64
 16  ax_min        1534 non-null   float64
 17  ay_min        1534 non-null   float64
 18  az_min        1534 non-null 

In [9]:
# TODO: Display the number of rows and columns  of all three datasets
# (1 mark)
print(f"'cv_set' has {len(cv_set.axes[0])} rows and {len(cv_set.axes[1])} columns")
print(f"'test_set1' has {len(test_set1.axes[0])} rows and {len(test_set1.axes[1])} columns")
print(f"'test_set2' has {len(test_set2.axes[0])} rows and {len(test_set2.axes[1])} columns")

'cv_set' has 1534 rows and 33 columns
'test_set1' has 783 rows and 33 columns
'test_set2' has 297 rows and 33 columns


In [10]:
# TODO: Display all columns in cv_set
# (1 mark)

for column_name in cv_set.columns.values:
  print(column_name)

ax_sum
ay_sum
az_sum
gx_sum
gy_sum
gz_sum
a_sum
g_sum
ax_var
ay_var
az_var
gx_var
gy_var
gz_var
a_var
g_var
ax_min
ay_min
az_min
gx_min
gy_min
gz_min
a_min
g_min
ax_max
ay_max
az_max
gx_max
gy_max
gz_max
a_max
g_max
motion_state


In all three dataframes, target vector is stored in the *motion_state* column. All other columns are features.

___

### 1.2 - Grid Search with Cross Validation (18 Marks)

___

#### 1.2.1 - Extract all features and target from cv_set

___

From the *cv_set*, load the target vector stored in the *motion_state* column into a new variable *y*

In [11]:
# TODO - Load target vector into y
# (1 mark)
y = cv_set['motion_state']

From *cv_set*, load all features, that is all columns except for the target vector, in a new dataframe variable *X*

In [12]:
# TODO - Load all features from cv_set to X
# (2 marks)
X = cv_set.drop("motion_state", axis='columns')
X.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,-0.003929,-0.093081,9.789349,-0.002649,0.001406,-0.001391,9.797418,0.022554,8.046094e-07,1.319665e-07,...,0.023871,0.000003,0.001496,0.000544,0.024992,0.000110,0.000075,0.000078,0.024997,0.000187
1,-0.000966,-0.092417,9.789095,-0.001806,0.000103,-0.001943,9.798815,0.021731,1.032619e-06,1.073067e-07,...,0.024153,0.000006,0.001865,0.000615,0.024828,0.000124,0.000098,0.000090,0.024829,0.000144


___

#### 1.2.2 - Configure and Perform Grid Search with Cross-Validation

___

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

The dataset *(X, y)*, must be split into training/validation *(X_train, y_train)* and testing *(X_test, y_test)* sets. Use the `train_test_split` function from sklearn to create the split, where  **30%** of data is used for testing. Set the `random_state=42`, and `stratify=y`.  

In [14]:
# TODO - Perform test/train split
# (1 mark)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

Initialize a `Pipeline`, consisting of a *scaler* and a *classifer*, set the default values to *StandardScaler()*, and *SVC()*, respectively.

In [15]:
# TODO - Initialize Pipeline into variable pipe
# (2 marks)

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', SVC())
])

Define the grid search configuration (*param_grid*) for training and hyper parameter tuning of the *support vector classifier*. Tune, the hyper-parameter *C*, over the interval `C=[0.01, 0.1, 1 ,10,100]`, *gamma*, over the interval `gamma=[0.001, 0.01, 0.1,1]`, and use the *radial basis function* as the kernel. For the *scaler* parameter, test with the `StandardScaler`, `RobustScaler`, and `None` options.

In [16]:
# TODO - Setup grid set
# (4 marks)
param_grid = {
    'classifier__C': [0.01, 0.1, 1, 10, 100],
    'classifier__gamma': [0.001, 0.01, 0.1, 1],
    'scaler': [StandardScaler(), RobustScaler(), None]
}

In [17]:
# Perform the grid search to train model and tune hyper parameters
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier', SVC())]),
             param_grid={'classifier__C': [0.01, 0.1, 1, 10, 100],
                         'classifier__gamma': [0.001, 0.01, 0.1, 1],
                         'scaler': [StandardScaler(), RobustScaler(), None]},
             return_train_score=True, scoring='accuracy')

___

#### 1.2.3 - Visualize Grid Search Results

___

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [18]:
# TODO: Print the results from the grid search
# (2 marks)
print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation train score: {:.2f}".format(grid.best_score_))
print("Best cross-validation validation score: {:.2f}".format(grid.cv_results_['mean_test_score'][grid.best_index_]))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

Best params:
{'classifier__C': 100, 'classifier__gamma': 0.01, 'scaler': RobustScaler()}

Best cross-validation train score: 0.95
Best cross-validation validation score: 0.95
Test-set score: 0.96


Display the confusion matrix and print the classification report for the trained model. Use the *(X_test, y_test)* dataset. **Hint** - You can use your code from assignment 3.

In [19]:
# TODO - Display confusion matrix and print classification report for (X_test, y_test)
# (2 marks)
from sklearn.metrics import confusion_matrix, classification_report
y_pred = grid.predict(X_test)

conf_mat = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_mat)

report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

Confusion Matrix:
 [[153   2   0]
 [ 10 192   5]
 [  0   3  96]]
Classification Report:
               precision    recall  f1-score   support

        rest       0.94      0.99      0.96       155
    straight       0.97      0.93      0.95       207
        turn       0.95      0.97      0.96        99

    accuracy                           0.96       461
   macro avg       0.95      0.96      0.96       461
weighted avg       0.96      0.96      0.96       461



___

#### 1.2.4 - Questions

1. Comment on the fit of the model to the dataset? (2 marks)
2. In the grid search, are the intervals used for *C* and *gamma* hyperparameters suitable? Why or why not? (2 marks)

#### 1.2.4 - Answers
1. The model shows a good fit to the training dataset as indicated by high cross validation training score (0.95), cross-validation validation score (0.95), and test-set score (0.96)
Further, the confusion matrix shows very less missclassification across all classes("straight", "rest", "trurn").
However this excellent fit does not guarantee a great generalising ability to other unseen data sets.
2.  The values of C ([0.01, 0.1, 1, 10, 100]) span a wide range, covering both small value of C and large value of C.
It it worth noting that exploring even larger values of C might be worth considering. The current range might miss an even better C value.
Same way, the gamma values ([0.001, 0.01, 0.1, 1]) covers a good range of values.
The optimal value of 0.01 suggests that the model prefers a less sensitive kernel, which is appropriate for many datasets.
Researching values smaller than 0.001 could be wise only if the best C value is quite high.
To sum it up, intervals used for C and gamma hyperparameters in grid search are reasonably suitable given the results till now, and identifies a well-performing model for this specific dataset.

___

### 1.3 - Model Evaluation on Test Datasets 1 and 2 (9 Marks)

___

#### 1.3.1 - Extract all features and targets from test datasets

Next, the model trained in 1.2 will be tested on independent datasets, that is *test_set1* and *test_set2*. <br>
The test_sets must be seperated into features (Xs) and targets (ys).

In [20]:
# TODO - Load the target vector from test_set1 into y1, and all features from test_set1 into X1
# (1 marks)
y1 = test_set1['motion_state']
X1 = test_set1.drop("motion_state", axis='columns')
X1.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,0.264465,0.089904,9.786930,-0.002305,0.000218,-0.001143,9.801661,0.025634,0.000001,7.749919e-08,...,0.024056,0.000003,0.002506,0.000845,0.024884,0.000155,0.000078,0.000069,0.024886,0.000170
1,0.264300,0.088882,9.788127,-0.001865,-0.000523,-0.001999,9.802481,0.025218,0.000001,9.243000e-08,...,0.024084,0.000007,0.002427,0.000949,0.024881,0.000201,0.000095,0.000076,0.024883,0.000201


In [21]:
# TODO - Load the target vector from test_set2 into y2, and all features from test_set2 into X2
y2 =  test_set2['motion_state']
X2 = test_set2.drop("motion_state", axis='columns')
X2.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,-0.046400,0.305907,9.791451,-0.002184,0.001311,-0.001901,9.800683,0.012437,0.000002,3.020959e-07,...,0.048406,0.000007,0.002575,0.002993,0.049701,0.000094,0.000115,0.000080,0.049764,0.000142
1,-0.050073,0.311947,9.785619,-0.003369,0.001898,-0.002051,9.797741,0.013805,0.000003,6.859937e-07,...,0.047696,0.000013,0.003880,0.004106,0.050477,0.000140,0.000140,0.000083,0.050763,0.000151


____

#### 1.3.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [22]:
# TODO - Print test score accuracies for test_set1 and test_set2
# (1 marks)
y1_pred = grid.predict(X1)
y2_pred = grid.predict(X2)
print("test_set1 score: {:.2f}".format(grid.score(X1, y1)))
print("test_set2 score: {:.2f}".format(grid.score(X2, y2)))

test_set1 score: 0.92
test_set2 score: 0.25


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [23]:
# TODO - Display confusion matrix and print classification report for (X1, y1)
# (1 marks)

conf_mat = confusion_matrix(y1, y1_pred)
print("Confusion Matrix:\n", conf_mat)

report = classification_report(y1, y1_pred)
print("Classification Report:\n", report)

Confusion Matrix:
 [[200  31   0]
 [  7 387  17]
 [  0   7 134]]
Classification Report:
               precision    recall  f1-score   support

        rest       0.97      0.87      0.91       231
    straight       0.91      0.94      0.93       411
        turn       0.89      0.95      0.92       141

    accuracy                           0.92       783
   macro avg       0.92      0.92      0.92       783
weighted avg       0.92      0.92      0.92       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [24]:
# TODO - Display confusion matrix and print classification report for (X2, y2)

conf_mat = confusion_matrix(y2, y2_pred)
print("Confusion Matrix:\n", conf_mat)

report = classification_report(y2, y2_pred)
print("Classification Report:\n", report)

Confusion Matrix:
 [[  0   0  16]
 [  0   0 206]
 [  0   0  75]]
Classification Report:
               precision    recall  f1-score   support

        rest       0.00      0.00      0.00        16
    straight       0.00      0.00      0.00       206
        turn       0.25      1.00      0.40        75

    accuracy                           0.25       297
   macro avg       0.08      0.33      0.13       297
weighted avg       0.06      0.25      0.10       297



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


___

#### 1.3.3 - Questions

**Based on results so far**
1. Comment on the model's performances on the datasets *(X_test, y_test)*, *(X1, y1)*, *(X2, y2)*? A significant difference in performance should be observed, what is this an example of? (3 marks)
1. Provide and explain potential causes for the observed behaviour in performance? (3 marks)

#### 1.3.3 - Answers
1. For dataset X_test, y_test) =>


*  On this dataset, the model performs very well, achieving high sccuracy (0.96)
* This implies strong fit to the data distributionfor this data set.


For (X1, y1) =>

* The performance degrades on this dataset, the accuracy dropped to 0.92.
* While this is still good, the decrease indicates the model's generalising capability is slightly limited when facing a different dataset.


For (X2, y2) =>

* The performance on this dataset drops drastically to 0.25. This indicates its failure to generalize.

This significant difference in performance, despite having high performance on (X_test, y_test) is a clear example of overfitting.

2. Potential causes for this observed behaviour =>

a. High complexity :
	The model was trained with all 32 features, which might have lead this model to learn the the specific nuances of the training data rather than learning generalized patterns.
	Resulting into an "over-trained" model.
  
b. Data distribution discrepancies :
	The datasets (X_test, y_test), (X1, y1), and (X2, y2) likely have different data distributions. So it might be possible that the features might be having different mean and variances.
	The model being tuned to (X_test, y_test), fails to adapt when presented data from different distribution.

___

## Section 2 - Classification Using *Selected Features* Derived From Inertial Sensor Data (12 Marks)

In this second section, an *SVC* model will be trained and tuned using grid search, however, only *selected features* will be used.

___

### 2.1 Grid Search with Cross Validation (5 Marks)

___

#### 2.1.1 Extract *selected* features and target from *cv_set*

From the *cv_set* dataframe, extract the selected features, listed below, and load them in a new dataframe *X*.
* *ax_var* - variance of accelerometer readings in x
* *ay_var* - variance of accelerometer readings in y
* *az_sum* - sum of accelerometer z readings
* *a_sum* - sum of the norm of accelerometer x, y, z readings
* *gz_var* - variance of gyroscope reading in z

In [25]:
# TODO - Load selected features from cv_set to X
# (1 marks)
X = cv_set[['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gz_var']]
X.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,8.046094e-07,1.319665e-07,9.789349,9.797418,7.446916e-10
1,1.032619e-06,1.073067e-07,9.789095,9.798815,6.983481e-10


In [26]:
# TODO - Load motion_state column (target vector) into y

y = cv_set['motion_state']

___

#### 2.1.2 - Configure and Perform Grid Search with Cross-Validation

Follow the steps from **1.2.2** and perform model training and tuning using the selected feature set.

In [27]:
# TODO - Perform model training and tuning using `GridSearchCV`

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', SVC())
])


param_grid = {
    'classifier__C': [0.01, 0.1, 1, 10, 100],
    'classifier__gamma': [0.001, 0.01, 0.1, 1],
    'scaler': [StandardScaler(), RobustScaler(), None]
}

grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier', SVC())]),
             param_grid={'classifier__C': [0.01, 0.1, 1, 10, 100],
                         'classifier__gamma': [0.001, 0.01, 0.1, 1],
                         'scaler': [StandardScaler(), RobustScaler(), None]},
             scoring='accuracy')

___

#### 2.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [28]:
# TODO: Print the results from the grid search

print("Best params:\n{}\n".format(grid_search.best_params_))
print("Best cross-validation train score: {:.2f}".format(grid_search.best_score_))
print("Best cross-validation test score: {:.2f}".format(grid_search.cv_results_['mean_test_score'][grid_search.best_index_]))
print("Test-set score: {:.2f}".format(grid_search.score(X_test, y_test)))

Best params:
{'classifier__C': 100, 'classifier__gamma': 0.01, 'scaler': RobustScaler()}

Best cross-validation train score: 0.82
Best cross-validation test score: 0.82
Test-set score: 0.83


Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [29]:
# TODO - Display confusion matrix and print classification reports


y_pred = grid_search.predict(X_test)

conf_mat = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_mat)

report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

Confusion Matrix:
 [[154   1   0]
 [ 20 173  14]
 [  1  44  54]]
Classification Report:
               precision    recall  f1-score   support

        rest       0.88      0.99      0.93       155
    straight       0.79      0.84      0.81       207
        turn       0.79      0.55      0.65        99

    accuracy                           0.83       461
   macro avg       0.82      0.79      0.80       461
weighted avg       0.82      0.83      0.82       461



___

#### 2.1.4 - Question


1. Consider the differences between the performance on the training/cross-validation dataset for section 1 (1.2.3) and section 2 (2.1.3). Which model is a better fit based on *just these two evaluations*? (4 marks)

#### 2.1.4 Answer
1.Based on the evaluation in section 1 (1.2.3) and section 2 (2.1.3), Model in section 1 is better fir for the dataset.
This is because of the below given reasons:

a. Model in section 1 has a significantly higher accuracy (0.96) compared to Model in section 2 (0.83)

b. In section 1 has higher precision and recall across all three classes ("straight", "rest", "turn") than the model trained in this section.
   This suggests that in section 1 (1.2.3), model makes fewer false positive and false negative classifications.

___

### 2.2 - Model Evaluation on Test Datasets 1 and 2 (7 Marks)

___

#### 2.2.1 - Extract selected features and targets from test datasets

The model trained in 2.1 is to be tested on the independent datasets, that is test_set1 and test_set2.
Seperate the testsets into the selected features and target vectors *(X1, y1)* and (X2,)

In [30]:
# TODO - Load the target vector from test_set1 into y1, and the selected features from test_set1 into X1
# (1 marks)

y1 = test_set1['motion_state']
X1 = test_set1[['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gz_var']]

In [31]:
# TODO - Load the target vector from test_set2 into y2, and the selected features from test_set2 into X2

y2 = test_set2['motion_state']
X2 = test_set2[['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gz_var']]

___

#### 2.2.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [32]:
# TODO - Print test score accuracies for test_set1 and test_set2
print("test_set1 score: {:.2f}".format(grid_search.score(X1, y1)))
print("test_set2 score: {:.2f}".format(grid_search.score(X2, y2)))

test_set1 score: 0.83
test_set2 score: 0.84


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [33]:
# TODO - Display confusion matrix and print classification report for (X1, y1)
y_pred = grid_search.predict(X1)

conf_mat = confusion_matrix(y1, y_pred)
print("Confusion Matrix:\n", conf_mat)

report = classification_report(y1, y_pred)
print("Classification Report:\n", report)

Confusion Matrix:
 [[229   2   0]
 [ 26 336  49]
 [  0  57  84]]
Classification Report:
               precision    recall  f1-score   support

        rest       0.90      0.99      0.94       231
    straight       0.85      0.82      0.83       411
        turn       0.63      0.60      0.61       141

    accuracy                           0.83       783
   macro avg       0.79      0.80      0.80       783
weighted avg       0.83      0.83      0.83       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [34]:
# TODO - Display confusion matrix and print classification report for (X2, y2)
y_pred = grid_search.predict(X2)

conf_mat = confusion_matrix(y2, y_pred)
print("Confusion Matrix:\n", conf_mat)

report = classification_report(y2, y_pred)
print("Classification Report:\n", report)

Confusion Matrix:
 [[ 15   1   0]
 [  0 178  28]
 [  0  18  57]]
Classification Report:
               precision    recall  f1-score   support

        rest       1.00      0.94      0.97        16
    straight       0.90      0.86      0.88       206
        turn       0.67      0.76      0.71        75

    accuracy                           0.84       297
   macro avg       0.86      0.85      0.85       297
weighted avg       0.85      0.84      0.84       297



___

#### 2.2.3 Questions

1. For section 2, compare between the model's performance on the training/validation (2.1) set to the the performance on the two testing sets (2.2). How does the model fit? (3 marks)
1. Compare the models' performances and fits between section 1 and section 2, and consider all three datasets. (3 marks)

#### 2.2.3 Answers
1.

Although not perfect, this model has a consistent performance on all three datasets.

For Training/Validation Set:

* Accuracy: 0.83
* Precision: Ranging from 0.79 to 0.88 across classes.
* Recall: Ranging from 0.55 to 0.99 across classes, showing notable variation across classes. The "turn" class shows lower recall (0.55).   

For test_set1:

* Accuracy: 0.83 (identical to training)
* Precision: Ranges from 0.63 to 0.90 across classes.
* Recall: Ranges from 0.60 to 0.99 across classes. The "turn" class again has low recall (0.60).

For test_set2:

* Accuracy: 0.84 (slightly better than training)
* Precision: Ranges from 0.67 to 1.00 across classes.
* Recall: Ranges from 0.76 to 0.94 across classes. The "turn" class has better recall (0.76) than in the other sets.

The model accuracy is consistent throught training/validation and two test sets indicates that the model generalizes quite well.
Overall, in this section, the model displays a decent fit, with quite well generalization to unseen data.
However, the lower recall for the "turn" class suggests that further model tuning is required to improve classification for "turn" class.

2.

Both the models in section 1 and 2 shows a noticeable difference between their performance and fit.

In section 1, the model shows high accuracy, poor generalization and high variance which is evident from the varying accuracy across all three datasets.

In section 2, the model shows moderate accuracy, consistent generalization and low variance.

Section 1 (Model trained with all features) =>

* Training Accuracy : 0.96
* test_set1 Accuracy : 0.92
* Test Set 2 Accuracy : 0.25
* Overfiting: high

Section 2 (Model trained with selected features) =>

* Training Accuracy : 0.83
* test_set1 Accuracy : 0.83
* test_set2 Accuracy : 0.84
* Overfiting: low

____

## Section 3 - Classification Using *Group Selected Features* From Inertial Sensor Data (10 Marks)

Once again, an *SVC* model is to be trained and tuned using grid search. However, you must select which features to use from the ones provided in *cv_set*.  

____

### 3.1 Grid Search with Cross Validation (4 Marks)

___

#### 3.1.1 Extract *Group Selected* features and target from *cv_set*

**Question**
1. Please list the features that you have selected and explain *why you have chosen* these features? (4 marks)



**Answer**

1.

Selected features =>

* ax_var, ay_var, az_var: Variance of acceleration in the x, y, and z axes.
* gz_var: Variance of gyroscope data in the z-axis.
* gz_sum, ay_sum, g_sum: Sum of gyroscopic data in the z-axis, acceleration in the y-axis, and the 2-norm of the gyroscope data (g_), respectively.
* gz_min, gz_max: Minimum and maximum values of gyroscopic data in the z-axis.
* ax_min, ax_max, ay_min, ay_max: Minimum and maximum values of acceleration in the x and y axes.

Rationale for feature selection as per our understanding =>

1. Prioritizing Strong Signals :
	The accelerometer data in the x and y directions and gyroscope data in the z direction are strong indicators of speed changes and turns.
	Therefore, we include variance (ax_var, ay_var, gz_var) to capture the dynamics of these movements.
	As mentioned in the video lecture, variance is less prone to bias according is robust to noise.
2.  Addressing Noisy Signals :
	The accelerometer z-axis is identified as noisy due to the influence of gravity and bounce. While we use the variance (az_var) to capture variability, we avoid using the mean/sum since that is bias prone.
3. Gyroscope z-axis :
   The gyroscope z-axis is a strong indicator of turn. So, we've included variance (gz_var), minimum (gz_min), maximum (gz_max), and sum (gz_sum) to capture the variability and the range of rotational motion during turns.
   The sum is also used, because while it can be affected by bias, this effect will be minimal compared to the large signal due to a turn.
4. The min and max values (ax_min, ax_max, ay_min, ay_max, gz_min, gz_max) are used as they provide additional information about the range of motion, which can be useful in classifying between different driving motions.

In [36]:
# TODO - Load group selected features from cv_set to X

X = cv_set[['ax_var', 'ay_var', 'gz_var', 'gz_sum', 'az_var', 'gz_min', 'gz_max', 'ax_min', 'ax_max', 'ay_min', 'ay_max', 'ay_sum', 'g_sum']]
X.head(2)

,ax_var,ay_var,gz_var,gz_sum,az_var,gz_min,gz_max,ax_min,ax_max,ay_min,ay_max,ay_sum,g_sum
0,8.046094e-07,1.319665e-07,7.446916e-10,-0.001391,3.267073e-08,-0.000082,0.000078,-0.001539,0.001496,-0.001109,0.000544,-0.093081,0.022554
1,1.032619e-06,1.073067e-07,6.983481e-10,-0.001943,1.529967e-08,-0.000090,0.000090,-0.001752,0.001865,-0.001025,0.000615,-0.092417,0.021731


In [38]:
# TODO - Load motion_state column (target vector) into y
y = cv_set['motion_state']

___

#### 3.1.2 - Configure and Perform Grid Search with Cross-Validation

Follow the steps from **1.2.2** and perform model training and tuning using the group selected feature set.

In [39]:
# TODO - Perform model training and tuning using `GridSearchCV`

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', SVC())
])


param_grid = {
    'classifier__C': [0.01, 0.1, 1, 10, 100],
    'classifier__gamma': [0.001, 0.01, 0.1, 1],
    'scaler': [StandardScaler(), RobustScaler(), None]
}

grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier', SVC())]),
             param_grid={'classifier__C': [0.01, 0.1, 1, 10, 100],
                         'classifier__gamma': [0.001, 0.01, 0.1, 1],
                         'scaler': [StandardScaler(), RobustScaler(), None]},
             scoring='accuracy')

___

#### 3.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [41]:
# TODO: Print the results from the grid search
print("Best params:\n{}\n".format(grid_search.best_params_))
print("Best cross-validation train score: {:.2f}".format(grid_search.best_score_))
print("Best cross-validation test score: {:.2f}".format(grid_search.cv_results_['mean_test_score'][grid_search.best_index_]))
print("Test-set score: {:.2f}".format(grid_search.score(X_test, y_test)))

Best params:
{'classifier__C': 100, 'classifier__gamma': 0.01, 'scaler': RobustScaler()}

Best cross-validation train score: 0.96
Best cross-validation test score: 0.96
Test-set score: 0.97


Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [43]:
# TODO - Display confusion matrix and print classification report

y_pred = grid_search.predict(X_test)

conf_mat = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:\n", conf_mat)

report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

Confusion Matrix:
 [[153   2   0]
 [ 10 195   2]
 [  0   2  97]]
Classification Report:
               precision    recall  f1-score   support

        rest       0.94      0.99      0.96       155
    straight       0.98      0.94      0.96       207
        turn       0.98      0.98      0.98        99

    accuracy                           0.97       461
   macro avg       0.97      0.97      0.97       461
weighted avg       0.97      0.97      0.97       461



___

### 3.2 - Model Evaluation on Test Datasets 1 and 2 (6 Marks)

___

#### 3.2.1 - Extract group selected features and targets from test datasets

The model trained in 3.1 is to be tested on the independent datasets, that is test_set1 and test_set2.
Seperate the testsets into group selected features and target vectors *(X1, y1)* and *(X2,y2)*

In [46]:
# TODO - Load the target vector from test_set1 into y1, and the group selected features from test_set1 into X1

y1 = test_set1['motion_state']
X1 = test_set1[['ax_var', 'ay_var', 'gz_var', 'gz_sum', 'az_var', 'gz_min', 'gz_max', 'ax_min', 'ax_max', 'ay_min', 'ay_max', 'ay_sum', 'g_sum']]
# X1 = test_set1[['ax_var', 'ay_var', 'ax_min', 'ay_min', 'ax_max', 'ay_max', 'gz_var', 'gz_min', 'gz_max']]
X1.head(2)

,ax_var,ay_var,gz_var,gz_sum,az_var,gz_min,gz_max,ax_min,ax_max,ay_min,ay_max,ay_sum,g_sum
0,0.000001,7.749919e-08,7.727190e-10,-0.001143,3.920063e-08,-0.000102,0.000069,-0.001191,0.002506,-0.000473,0.000845,0.089904,0.025634
1,0.000001,9.243000e-08,8.871433e-10,-0.001999,4.112300e-08,-0.000094,0.000076,-0.001104,0.002427,-0.000408,0.000949,0.088882,0.025218


In [48]:
# TODO - Load the target vector from test_set2 into y2, and the group selected features from test_set2 into X2
y2 = test_set2['motion_state']
X2 = test_set2[['ax_var', 'ay_var', 'gz_var', 'gz_sum', 'az_var','gz_min', 'gz_max', 'ax_min', 'ax_max', 'ay_min', 'ay_max', 'ay_sum', 'g_sum']]
# X2 = test_set2[['ax_var', 'ay_var', 'ax_min', 'ay_min', 'ax_max', 'ay_max', 'gz_var', 'gz_min', 'gz_max']]
X2.head(2)

,ax_var,ay_var,gz_var,gz_sum,az_var,gz_min,gz_max,ax_min,ax_max,ay_min,ay_max,ay_sum,g_sum
0,0.000002,3.020959e-07,1.405038e-09,-0.001901,6.324738e-08,-0.000122,0.000080,-0.002970,0.002575,0.000257,0.002993,0.305907,0.012437
1,0.000003,6.859937e-07,1.164381e-09,-0.002051,3.110085e-07,-0.000098,0.000083,-0.003824,0.003880,-0.000567,0.004106,0.311947,0.013805


___

#### 3.2.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [52]:
# TODO - Print test score accuracies for test_set1 and test_set2

print("test_set1 score: {:.2f}".format(grid_search.score(X1, y1)))
print("test_set2 score: {:.2f}".format(grid_search.score(X2, y2)))

test_set1 score: 0.97
test_set2 score: 0.79


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [54]:
# TODO - Display confusion matrix and print classification report for (X1, y1)
y_pred = grid_search.predict(X1)

conf_mat = confusion_matrix(y1, y_pred)
print("Confusion Matrix:\n", conf_mat)

report = classification_report(y1, y_pred)
print("Classification Report:\n", report)

Confusion Matrix:
 [[228   3   0]
 [  7 398   6]
 [  0   8 133]]
Classification Report:
               precision    recall  f1-score   support

        rest       0.97      0.99      0.98       231
    straight       0.97      0.97      0.97       411
        turn       0.96      0.94      0.95       141

    accuracy                           0.97       783
   macro avg       0.97      0.97      0.97       783
weighted avg       0.97      0.97      0.97       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [56]:
# TODO - Display confusion matrix and print classification report for (X2, y2)
y_pred = grid_search.predict(X2)

conf_mat = confusion_matrix(y2, y_pred)
print("Confusion Matrix:\n", conf_mat)

report = classification_report(y2, y_pred)
print("Classification Report:\n", report)

Confusion Matrix:
 [[  7   9   0]
 [  0 152  54]
 [  0   0  75]]
Classification Report:
               precision    recall  f1-score   support

        rest       1.00      0.44      0.61        16
    straight       0.94      0.74      0.83       206
        turn       0.58      1.00      0.74        75

    accuracy                           0.79       297
   macro avg       0.84      0.73      0.72       297
weighted avg       0.86      0.79      0.79       297



___

#### 3.2.3 Questions

1. Based on the observations across all three datasets, compare the performance of the model trained in this section to: (3 marks)
    1. Model trained in Section 1 using *all features*.
    1. Model trained in Section 2 using *selected features*.

2. Please proivde potential explanations for the differences observed in models' performances across the three sections. (3 marks)

#### 3.2.3 Answers
1.
1.

____

## Section 4 - Classification Using Principal Components of Features From Inertial Sensor Data (10 Marks)

____

### 4.1 Principal Component Analysis (PCA) Search with Cross Validation (4 Marks)

___

#### 4.1.1 Extract *ALL* features and target from *cv_set*

In [ ]:
# TODO - Load all features from cv_set to X
# X =
X.head(2)

In [ ]:
# TODO - Load motion_state column (target vector) into y
# y =

___

#### 4.1.2 - Configure and Perform Grid Search with Cross-Validation

Similar to the steps from **1.2.2** and perform model training and tuning for an `SVC` model. Perform the `train_test_split`, initiate a `Pipeline`, then define the *Parameters* for `GridSearchCV`. Finally, perfrom the grid search.

In [ ]:
# TODO - Perform model training and tuning using `GridSearchCV`
# X_train, X_test, y_train, y_test =

In the `Pipeline` add an entry *'pca'* to apply `PCA`.

In [ ]:
# Perform any missing imports

# TODO - Initiate the pipeline
# (1 marks)
# pipe =

Similar to previous sections, define the grid search configuration (*param_grid*). Add *n_components* hyper parameter tuning for PCA, and search over the values `n_components=[4,7,11,14,16]`.  

In [ ]:
# TODO - Define the parameter grid for your model
# (3 marks)
# param_grid =

In [ ]:
# Perform the grid search to train model and tune hyper parameters
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

___

#### 4.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [ ]:
# TODO: Print the results from the grid search

#print("Best params:\n{}\n".format(???))
#print("Best cross-validation train score: {:.2f}".format(???))
#print("Best cross-validation validation score: {:.2f}".format(???))
#print("Test-set score: {:.2f}".format(???)))

Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [ ]:
# TODO - Display confusion matrix and print classification report

___

### 4.2 - Model Evaluation on Test Datasets 1 and 2 (6 Marks)

___

#### 4.2.1 - Extract group selected features and targets from test datasets

In [ ]:
# TODO - Load the target vector from test_set1 into y1, and all features from test_set1 into X1
# y1 =
# X1 =
X1.head(2)

In [ ]:
# TODO - Load the target vector from test_set2 into y2, and all features from test_set2 into X2
# y2 =
# X2 =
X2.head(2)

___

#### 4.2.2 - Visualize Results

In [ ]:
# TODO - Print test score accuracies for test_set1 and test_set2
#print("test_set1 score: {:.2f}".format(???))
#print("test_set2 score: {:.2f}".format(???))

In [ ]:
# TODO - Display confusion matrix and print classification report for (X1, y1)

Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [ ]:
# TODO - Display confusion matrix and print classification report for (X2, y2)

___

#### 4.2.3 - Questions

1. Based on the observations across all three datasets, compare the performance of the model with PCA, trained in this section to: (4 marks)
    * Model trained in Section 1 using *all features*.
    * Model trained in Section 2 using *selected features*.
    * Model trained in Section 3 using *group selected features*

2. Please provide potential explanations for the differences observed in models' performances across between section 1 and 4. (2 marks)

#### 4.2.3 - Answers

____

## Section 5 - Classification Using *Pre Selected Features* Derived From Inertial Sensor Data (10 Marks)

___

Now, a second classifer is to be trained and tuned using grid search. The **selected features from section 2** must be used. You may choose from one of the algorithms discussed that class listed below.
1. Random Forest
1. K-Nearest Neighbors
1. Gradient Boosting Machines
1. Neural Networks

____

### 5.1 Grid Search with Cross Validation (6 Marks)

___

#### 5.1.1 Extract *Selected* features and target from *cv_set*

**Questions**
1. Please list the ML model that you have selected . And explain *why you have chosen* this model? (2 marks)

**Answer**
1.

In [ ]:
# TODO - Load group selected features from cv_set to X
# X =
X.head(2)

In [ ]:
# TODO - Load motion_state column (target vector) into y
# y =

___

#### 5.1.2 - Configure and Perform Grid Search with Cross-Validation

Similar to the steps from **1.2.2** and perform model training and tuning using for the model selected. Perform the `train_test_split`, initiate a `Pipeline`, then define the *Parameters* for `GridSearchCV`. Finally, perfrom the grid search.

In [ ]:
# TODO - Perform train/test split

In [ ]:
# Perform any missing imports

# TODO - Initiate the pipeline
# (1 mark)
# pipe =

In [ ]:
# TODO - Define the parameter grid for your model
# (3 marks)
# param_grid =

In [ ]:
# TODO - Perform the grid search
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

___

#### 5.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [ ]:
# TODO: Print the results from the grid search

#print("Best params:\n{}\n".format(???))
#print("Best cross-validation train score: {:.2f}".format(???))
#print("Best cross-validation validation score: {:.2f}".format(???))
#print("Test-set score: {:.2f}".format(???)))

Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [ ]:
# TODO - Display confusion matrix and print classification report

___

### 5.2 - Model Evaluation on Test Datasets 1 and 2 (4 Marks)

___

#### 5.2.1 - Extract selected features and targets from test datasets

The model trained in 5.1, is to be tested on the independent datasets, that is test_set1 and test_set2.
Seperate the testsets into features and target vectors *(X1, y1)* and *(X2,y2)*

In [ ]:
# TODO - Load the target vector from test_set1 into y1, and the selected features from test_set1 into X1
# y1 =
# X1 =
X1.head(2)

In [ ]:
# TODO - Load the target vector from test_set2 into y2, and the selected features from test_set2 into X2
# y2 =
# X2 =
X2.head(2)

___

#### 5.2.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [ ]:
# TODO - Print test score accuracies for test_set1 and test_set2
#print("test_set1 score: {:.2f}".format(???))
#print("test_set2 score: {:.2f}".format(???))

Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [ ]:
# TODO - Display confusion matrix and print classification report for (X1, y1)

Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [ ]:
# TODO - Display confusion matrix and print classification report for (X2, y2)

___

#### 5.2.3 Question

1. For the model trainied in this section, compare its performance to the performance of SVC which was also trained with the same selected features (section 2). Which model fits better? Explain causes for any observed discrepancies? (4 marks)

#### 5.2.3 - Answer
1.


____